# TASK-5 Mookit Scraping
**I have used selenium and BeautifulSoup libraries to complete the task**

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup    
import urllib3 
import requests
import getpass
import pandas as pd

## Login to mookit
And preparing beutifulsoup objects for scraping

In [13]:
usrn = input("Enter your username for mookit:")
passw = getpass.getpass(prompt="Please enter your password:")
url_to_course = input("Enter url to course which you want to look at")
url_to_course = url_to_course.replace("/course","")

In [14]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
driver.get('https://hello.iitk.ac.in/index.php/user/login')
username = driver.find_element_by_id("edit-name")
password = driver.find_element_by_id("edit-pass")
username.send_keys(usrn)
password.send_keys(passw)
driver.find_element_by_id("edit-submit").click()
driver.get(url_to_course+"/#/home")

In [15]:
source = driver.page_source
soup = BeautifulSoup(source,'html.parser')

## Pulling Week information

In [16]:
weeks = []
data = soup.findAll('div',{"class":"weekWrapper"})
for dat in data:
    weeks.append(dat.get_text(strip=True))
print(weeks)

['Week 1', 'Week 2', 'Week 3']


In [17]:
weeklist = soup.findAll('ul',{"class":"weekList"})

In [18]:
lecture_list_items = []
for week_lec in weeklist:
    # print(str(week_lec))
    lecs = BeautifulSoup(str(week_lec),'html.parser')
    lis = lecs.findAll('li')
    print(len(lis))
    lecture_list_items.append(lis)

2
2
1


In [19]:
def remove_spaces(string):
    s = ' '.join(string.split())
    return s

## Finally pulling all info about lectures
**I have pulled lec name, link to lecture, and links to attachements here**

In [20]:
all_lecs_info = []
for a_week in lecture_list_items:
    weekly_lecs = []
    for one_item in a_week:
        lecs_details = []
        one_item = BeautifulSoup(str(one_item),'html.parser')

        title = one_item.find("span",{"class":"weekListItemTitle"})
        title = remove_spaces(title.text)
        lecs_details.append(title[1:])

        lecs_name = one_item.findAll("div",{"class":"lectureInfoBoxText"})
        lec_names = []
        for ln in lecs_name:
            lec_names.append(remove_spaces(ln.text))
        lecs_details.append(lec_names)

        lecs_length = one_item.findAll("div",{"class":"lectureInfoBoxLength"})
        lec_lenghts = []
        for ln in lecs_length:
            lec_lenghts.append(ln.text)
        lecs_details.append(lec_lenghts)

        lecs_link = one_item.find_all('a',{"_ngcontent-c2":""},href=True)
        lec_links = []
        for ln in lecs_link:
            lec_links.append(ln['href'])
        lecs_details.append(lec_links)
        # now lecs_details has 4 items: title, a list containing name of lectures, a list containing corresponding lengths, a list containing links to lectures
        weekly_lecs.append(lecs_details)

    all_lecs_info.append(weekly_lecs)
# print(all_lecs_info) 

    

## Coverting data to pandas dataframe and saving submission

In [21]:
c = ["Week",'heading','Name','lecture duration','link to lecture','link to attachements']
df = pd.DataFrame(columns=c)
for i,week in enumerate(all_lecs_info):
    weeknum = weeks[i]
    for head in week:
        heading = head[0]
        n = len(head[1])
        k = 0
        for i in range(n):
            lec_name = head[1][i]
            lec_dur = head[2][i]

            n_links = len(head[3])
            while k<n_links and head[3][k][2:6]=="inst":
                k+=1
            
            lec_link = url_to_course + "/" + head[3][k]
            lec_attach = ""
            k+=1
            j=0
            while k<n_links and head[3][k][0:4]=="http":
                if j==0:
                    lec_attach = lec_attach + head[3][k]
                    j=1
                else:
                    lec_attach = lec_attach + "\n" + head[3][k]
                k+=1
            df = df.append({c[0]:weeknum,c[1]:heading,c[2]:lec_name,c[3]:lec_dur,c[4]:lec_link,c[5]:lec_attach},ignore_index=True)
print(df)

     Week               heading       Name lecture duration  \
0  Week 1          Introduction  Lecture 1            00:00   
1  Week 1  Preparing an outline  Lecture 2            00:00   
2  Week 2             Reading 1  Lecture 3            00:00   
3  Week 2             Reading 2  Lecture 4            00:00   
4  Week 3           Note Making  Lecture 5            00:00   
5  Week 3           Note Making  Lecture 6            00:00   

                                  link to lecture link to attachements  
0   https://hello.iitk.ac.in/com20021/#/lecture/5                       
1   https://hello.iitk.ac.in/com20021/#/lecture/6                       
2  https://hello.iitk.ac.in/com20021/#/lecture/12                       
3  https://hello.iitk.ac.in/com20021/#/lecture/13                       
4  https://hello.iitk.ac.in/com20021/#/lecture/66                       
5  https://hello.iitk.ac.in/com20021/#/lecture/67                       


In [22]:
num_lecs = int(input("How many Last Lectures you are interested in? Total lectures are "+str(len(df.axes[0]))))
last_df = df.tail(num_lecs)
last_df.to_csv('lectures.csv',index=False)